<a href="https://colab.research.google.com/github/covercash2/deep/blob/master/Deep_Daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Restart after running this cell!**

You must run this cell and then restart and rerun everything for the PyTorch version to be correct. Otherwise the model will run but not produce any meaningful output.

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

In [ ]:
!pip install deep-daze --upgrade

In [ ]:
from tqdm import trange
from IPython.display import Image, display

from deep_daze import Imagine

TEXT = 'a room full of teeth' #@param {type:"string"}
NUM_LAYERS = 32 #@param {type:"number"}
SAVE_EVERY =  20#@param {type:"number"}
IMAGE_WIDTH = 512 #@param {type:"number"}
SAVE_PROGRESS = True #@param {type:"boolean"}
BATCH_SIZE = 8#@param {type: "number"}
LEARNING_RATE = 1e-5 #@param {type:"number"}
ITERATIONS = 300 #@param {type:"number"}
EPOCHS =  30#@param {type: "number"}

model = Imagine(
    text = TEXT,
    num_layers = NUM_LAYERS,
    save_every = SAVE_EVERY,
    image_width = IMAGE_WIDTH,
    batch_size = BATCH_SIZE,
    lr = LEARNING_RATE,
    iterations = ITERATIONS,
    save_progress = SAVE_PROGRESS,
    epochs = EPOCHS
)

for epoch in trange(EPOCHS, desc = 'epochs'):
    for i in trange(ITERATIONS, desc = 'iteration'):
        model.train_step(epoch, i)

        if i % model.save_every != 0:
            continue

        filename = TEXT.replace(' ', '_')
        image = Image(f'./{filename}.png')
        display(image)

# create a video from the sequence

In [ ]:
import os
import re
import glob
import cv2
import numpy as np

%cd /content

number_re = re.compile(r"/content/a_room_full_of_teeth.(\d+).png")

image_sequence = [] 

for filename in glob.iglob('/content/*.png'):
  # get the first match, 
  # i.e. the sequence number of the image,
  # as the new file name
  try:
    sequence_num = int(number_re.match(filename)[1])
    print(sequence_num)
    image_sequence.insert(sequence_num, filename)
  except TypeError as err:
    print(f"could not convert file: {err}")
 
img_array = []
for filename in image_sequence:
  print(filename)
  img = cv2.imread(filename)
  height, width, layers = img.shape
  size = (width,height)
  img_array.append(img)
 
 
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()